In [1]:
import pickle
import pprint
import json
import time
import pandas as pd

# Load Log

In [ ]:
# with open('EURUSD 01-04 to 18-06.txt', 'rb') as f:
#     logs = pickle.load(f)

In [ ]:
# out = open("EURUSD 01-04 to 18-06 websocket data.txt", "wt", encoding='utf-8')

# for log in logs:
    
#     for event in log:

#         x = json.loads(event["message"])["message"]
#         if "Network.webSocketFrameReceived" in x["method"] or "Network.webSocketFrameSent" in x["method"]:
#             pprint.pprint(x, stream=out)
        
# out.close()

In [2]:
# Utils

def split_sent_received(file):
    sent = []
    received = []
    for event in file:
        x = json.loads(event["message"])["message"]
        if "Network.webSocketFrameReceived" in x["method"]: 
            received.append(json.loads(x["params"]["response"]["payloadData"]))
        elif "Network.webSocketFrameSent" in x["method"]:
            sent.append(json.loads(x["params"]["response"]["payloadData"]))
    return sent, received

def search_by_name(list, keys):
    l = [element for element in list if element["name"] in keys]
    return l

def get_request_ids(list):
    d = {}
    for element in list:
        if element["name"] == "sendMessage" and element["msg"]["name"] == "get-candles":
            d[element["request_id"]] = {
                "active_id": element["msg"]["body"]["active_id"],
                "size": element["msg"]["body"]["size"],
            }
    return d

def get_candles(list):
    candles_dict = {}
    for element in list:
        if element["name"] == "candles":
            try:
                candles_dict[element["request_id"]] = pd.DataFrame(element["msg"]["candles"]).set_index("id")
            except:
                print(element)
    return candles_dict

def fill_assets_dict(request_ids, candles):
    from tqdm.notebook import tqdm
    assets = {}
    for request_id in tqdm(request_ids):
        active_id = request_ids[request_id]['active_id']
        size = request_ids[request_id]["size"]

        if active_id not in assets.keys():
            try:
                assets[active_id] = {
                    'name': binary_op_ids[active_id],
                    'data': {}
                }
            except:
                assets[active_id] = {
                    'name': '',
                    'data': {}
                }
        try:
            index = pd.MultiIndex.from_product([[size], candles[request_id].index.to_numpy()], names=['size', 'id'])
            candle = candles[request_id].set_index(index)
        except KeyError:
            print("KeyError:", request_id)
            
        if isinstance(assets[active_id]["data"], pd.DataFrame):
            assets[active_id]["data"] = assets[active_id]["data"].merge(candle, how='outer', on=candle.columns.values.tolist(), left_index=True, right_index=True)
        else:
            assets[active_id]["data"] = candle
    
    for key in assets:
        assets[key]['data'] = assets[key]['data'].sort_index()
    
    return assets

# def get_last_candle_gen(candle_generated_list):
#     # Get the last candle generated of a given id
    
#     assets = {}

#     for element in candle_generated_list:
        
#         active_id = element['msg']['active_id']
#         id_ = element['msg']['id']
        
#         candle_dict = {
#             'size': element['msg']['size'],
#             'from': element['msg']['from'],
#             'to': element['msg']['to'],
#             'open': element['msg']['open'],
#             'close': element['msg']['close'],
#             'min': element['msg']['min'],
#             'max': element['msg']['max'],
#             'volume': element['msg']['volume'],
#         } 
    
    
#         if active_id not in assets.keys():
#             assets[active_id] = {
#                 'data': {id_: candle_dict}
#             }
#         else:
#             assets[active_id]['data'][id_] = candle_dict
            
#     cols = ['size', 'from', 'to', 'open', 'close', 'min', 'max', 'volume']
    
#     for key in assets.keys():
        
#         df = assets[key]['data']
#         df = pd.DataFrame.from_dict(df, orient='index', columns=cols).reset_index()
#         index = pd.MultiIndex.from_frame(df[['size', 'index']], names=['size', 'id'])
#         assets[key]['data'] = df.set_index(index).drop(['index', 'size'], axis=1)
        
#     return assets

In [3]:
with open("EURNZD 01-04 to 18-06.txt", "rb") as f:
    l = pickle.load(f)
    
sent_list = []
received_list = []
for element in l:
    
    sent, received = split_sent_received(element)
    sent_list.append(sent)
    received_list.append(received)

sent_list2 = [element2 for element in sent_list for element2 in element]
received_list2 = [element2 for element in received_list for element2 in element]
sent_list = sent_list2
received_list = received_list2
del l, sent_list2, received_list2, sent, received

# Handling candles

In [4]:
# Get request_ids 
start = time.time()
request_id_list = get_request_ids(sent_list)
dt1 = time.time() - start

# Get the data
start = time.time()
get_candles_list = get_candles(received_list)   
dt2 = time.time() - start
dt1, dt2

(0.010987520217895508, 12.412444829940796)

In [5]:
binary_op_ids = { 7: 'AUDCAD', 943: 'AUDCHF', 101: 'AUDJPY', 944: 'AUDNZD', 99: 'AUDUSD', 107: 'CADCHF',
                106: 'CHFJPY', 108: 'EURAUD', 2: 'EURGBP', 4: 'EURJPY', 212: 'EURNZD', 1: 'EURUSD',
                104: 'GBPAUD', 102: 'GBPCAD', 103: 'GBPCHF', 3: 'GBPJPY', 947: 'GBPNZD', 5: 'GBPUSD', 8: 'NZDUSD',
                100: 'USDCAD', 72: 'USDCHF', 6: 'USDJPY', 168: 'USDNOK'}

# fill the assets dict
start = time.time()
assets = fill_assets_dict(request_id_list, get_candles_list)
dt3 = time.time() - start
dt3/60

  0%|          | 0/5302 [00:00<?, ?it/s]

21.972699193159738

In [6]:
assets.keys()

dict_keys([1, 212])

In [8]:
assets[212]['data'].to_csv('EURNZD.csv')